In [1]:
import os
from datetime import datetime
from selenium import webdriver
import urllib
#from xvfbwrapper import Xvfb
import http.server
import socketserver
from threading import Thread
import time
import zipfile
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [3]:
########################################
PROXY_HOST = ''  # rotating proxy or host
PROXY_PORT = 31112 # port
PROXY_USER = '' # username
PROXY_PASS = '' # password
################################

In [29]:
manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)


prefs = {
    'profile.default_content_setting_values': {
        #'cookies': 2, #'images': 2,
        'plugins': 2, 'popups': 2, 'geolocation': 2,
        'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2,
        'mouselock': 2, 'mixed_script': 2, 'media_stream': 2,
        'media_stream_mic': 2, 'media_stream_camera': 2,
        'protocol_handlers': 2,
        'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2,
        'push_messaging': 2, 'ssl_cert_decisions': 2,
        'metro_switch_to_desktop': 2,
        'protected_media_identifier': 2, 'app_banner': 2,
        'site_engagement': 2,
        'durable_storage': 2, 'favicon': 2
    }
}

In [30]:
def get_chromedriver(use_proxy=False, user_agent=None):
    #path = os.path.dirname(os.path.abspath(__file__))
    os.environ["DBUS_SESSION_BUS_ADDRESS"] = '/dev/null'
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-infobars")
    if use_proxy:
        pluginfile = 'proxy_auth_plugin.zip'

        with zipfile.ZipFile(pluginfile, 'w') as zp:
            zp.writestr("manifest.json", manifest_json)
            zp.writestr("background.js", background_js)
#        chrome_options.add_extension(pluginfile)
    if user_agent:
        chrome_options.add_argument('--user-agent=%s' % user_agent)
    # driver = webdriver.Chrome(chrome_options=chrome_options)
    ch_driver_path=r"/Users/saanvi/Development/Upwork/chromedriver"
    #driver = webdriver.Chrome('/home/centos/chromedriver', options=chrome_options)
    driver = webdriver.Chrome(ch_driver_path, options=chrome_options)
    return driver

In [31]:
def get_chromedriver(use_proxy=False, user_agent=None):
    #path = os.path.dirname(os.path.abspath(__file__))
    os.environ["DBUS_SESSION_BUS_ADDRESS"] = '/dev/null'
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-infobars")
    if use_proxy:
        pluginfile = 'proxy_auth_plugin.zip'

        with zipfile.ZipFile(pluginfile, 'w') as zp:
            zp.writestr("manifest.json", manifest_json)
            zp.writestr("background.js", background_js)
#        chrome_options.add_extension(pluginfile)
    if user_agent:
        chrome_options.add_argument('--user-agent=%s' % user_agent)
    # driver = webdriver.Chrome(chrome_options=chrome_options)
    ch_driver_path=r"/Users/saanvi/Development/Upwork/chromedriver"
    #driver = webdriver.Chrome('/home/centos/chromedriver', options=chrome_options)
    driver = webdriver.Chrome(ch_driver_path, options=chrome_options)
    return driver

In [45]:
driver = get_chromedriver(use_proxy=False)
driver.set_page_load_timeout(30)
driver.set_script_timeout(15)
link='https://www.5miles.com/ca/dallas_tx/cars-trucks_14#/p1'
driver.get(link)
driver.find_element_by_xpath('//*[@id="homeLayout"]').click()

driver.find_element_by_xpath('//*[@id="cookie-remind"]/div/div/div[1]/button').click()

In [17]:
source = driver.execute_script("return document.body.innerHTML")

In [46]:
#print(source)

In [43]:
driver.find_element_by_xpath('//*[@id="cookie-remind"]/div/div/div[1]/button').click()

In [44]:
driver.close()